# Fine-tuning GPT-4o to Write LinkedIn Posts (in my style)
## ABB #4 - Session 5

Code authored by: Shaw Talebi

### imports

In [1]:
import os
import csv
import json
import random

from openai import OpenAI
from dotenv import load_dotenv

In [2]:
# import sk from .env file
load_dotenv()

# connect to openai API
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

### Read data

In [3]:
# load csv of YouTube comments
idea_list = []
copy_list = []
media_list = []

with open('data/LI_posts.csv', mode ='r') as file:
    file = csv.reader(file)
    
    # read file line by line
    for line in file:
        # skip first line
        if line[0]=='Idea':
            continue
            
        # append comments and responses to respective lists
        idea_list.append(line[0])
        copy_list.append(line[1])
        media_list.append(line[2])

In [4]:
print(len(idea_list))
print(len(copy_list))
print(len(media_list))

63
63
63


### Create training examples

In [5]:
# construct training examples
example_list = []

system_prompt = "LinkedIn Post Writer for Shaw Talebi, AI educator and entrepreneur"

prompt_template = lambda idea_string : f"""Write a LinkedIn post based on the following idea:
{idea_string}

Include:
- A compelling opening line that hooks the reader
- Copy that expands upon the idea in valuable way
- A call to action or share relevant content

Output:
"""

for i in range(len(idea_list)):    
    system_dict = {"role": "system", "content": system_prompt}
    user_dict = {"role": "user", "content": prompt_template(idea_list[i])}
    assistant_dict = {"role": "assistant", "content": copy_list[i] + "\n\n--\nMedia: " + media_list[i]}
    
    messages_list = [system_dict, user_dict, assistant_dict]
    
    example_list.append({"messages": messages_list})

In [6]:
print(example_list[0]['messages'][0]['content'])
print(example_list[0]['messages'][1]['content'])
print(example_list[0]['messages'][2]['content'])

LinkedIn Post Writer for Shaw Talebi, AI educator and entrepreneur
Write a LinkedIn post based on the following idea:
3 types of AI Tik Tok

Include:
- A compelling opening line that hooks the reader
- Copy that expands upon the idea in valuable way
- A call to action or share relevant content

Output:

A problem with AI today is that it means different things to different people. 

This framework from Andrej Karpathy helped give me much more clarity 👇 

Software 1.0 = Rule-based software systems. Humans program computers to solve problems step-by-step. 

Software 2.0 = Computers program themselves by seeing examples (i.e. machine learning) 

Software 3.0 = Repurposing general-purpose ML models for specific use cases (i.e. GenAI + Foundation Models) 

But… what’s Software 4.0 going to be? 🤔

--
Media: Video


In [7]:
len(example_list)

63

### Create train/validation split

In [8]:
# randomly pick out validation examples
num_examples = 15
validation_index_list = random.sample(range(0, len(example_list)-1), num_examples)
validation_data_list = [example_list[index] for index in validation_index_list]

for example in validation_data_list:
    example_list.remove(example)

In [9]:
print(len(example_list))
print(len(validation_data_list))

48
15


In [10]:
# write examples to file
with open('data/train-data.jsonl', 'w') as train_file:
    for example in example_list:
        json.dump(example, train_file)
        train_file.write('\n')

with open('data/valid-data.jsonl', 'w') as valid_file:
    for example in validation_data_list:
        json.dump(example, valid_file)
        valid_file.write('\n')

### Upload data to OpenAI

In [11]:
train_file = client.files.create(
  file = open("data/train-data.jsonl", "rb"),
  purpose = "fine-tune"
)

valid_file = client.files.create(
  file = open("data/valid-data.jsonl", "rb"),
  purpose = "fine-tune"
)

### Fine-tune model

In [12]:
client.fine_tuning.jobs.create(
    training_file = train_file.id,
    validation_file = valid_file.id,
    suffix = "LI-post-writer",
    model = "gpt-4o-mini-2024-07-18",
    method={
    "type": "supervised",
    "supervised": {
      "hyperparameters": {
        "n_epochs": 3,
        "learning_rate_multiplier": 1.25
            }
        }
    }
)

FineTuningJob(id='ftjob-6cvQXjMkHQlpxRO4K9zw1DoI', created_at=1746137909, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier=1.25, n_epochs=3), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-KjWERyZ9WLUqIdrdMeJh4zC0', result_files=[], seed=601268377, status='validating_files', trained_tokens=None, training_file='file-GhxAiVpuMtCqZURvJGyo8W', validation_file='file-5eVsX9qZzTjfZQZxky7BGc', estimated_finish=None, integrations=[], metadata=None, method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier=1.25, n_epochs=3)), type='supervised'), user_provided_suffix='LI-post-writer', usage_metrics=None, shared_with_openai=False)

### Evaluate fine-tuned model

In [14]:
def generate_post(system_prompt, model_name, idea):
    response = client.chat.completions.create(
        model=model_name,
        messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt_template(idea)}
        ],
        temperature=1,
    )
    return response.choices[0].message.content

In [19]:
idea = "10 AI buzzwords explained for Entrepreneurs"

In [20]:
# GPT-4o (no fine-tuning)
model_name = "gpt-4o"
system_prompt_long = "You are an AI assistant helping Shaw Talebi, an AI educator and entrepreneur, craft LinkedIn posts. Your goal is to generate posts \
that reflect Shaw Talebi's voice: authoritative yet approachable, insightful yet concise. Shaw Talebi's posts aim to educate and inspire professionals \
in the tech and AI space. Focus on providing value, discussing new trends, or offering actionable advice, while keeping the tone professional but \
conversational. The target audience includes entrepreneurs, tech professionals, and decision-makers in AI and data science. Always ensure the post is \
relevant, engaging, and on-brand for Shaw Talebi's public persona."

# print(system_prompt_long, "\n--")
print(generate_post(system_prompt_long, model_name, idea))

🔍 Ever felt lost in the sea of AI jargon? 🤖 As entrepreneurs, understanding AI terms isn't just beneficial—it's essential for staying ahead.

Let's demystify 10 commonly used AI buzzwords that can transform your business strategies:

1. **Machine Learning**: Think of it as teaching computers to learn patterns from data to make informed decisions. It’s like training a chef who gets better with every dish they serve.

2. **Deep Learning**: A subset of machine learning using neural networks with vast layers; it analyzes high-dimensional data complexity. Picture it as the brain of AI—intense, sophisticated, and incredibly capable.

3. **Neural Networks**: Inspired by the human brain's network, these are systems of algorithms used to identify relationships in sets of data through a process that mimics human thought.

4. **Natural Language Processing (NLP)**: The interaction between AI and human language. It enables systems to read, decipher, and understand our language. Think of it like hav

In [21]:
# GPT-4o-mini (fine-tuned)
model_name = "ft:gpt-4o-mini-2024-07-18:shawhin-talebi-ventures-llc:li-post-writer:BSWitilL"

# print(system_prompt, "\n--")
print(generate_post(system_prompt, model_name, idea))

Can we ever really cut through the jargon of new technologies? 

Here are 10 AI buzzwords that I have defined (for myself) over the last year… 

1) Foundation Model: A large AI model that has been trained on a general task and can be customized for specific applications using “fine-tuning”.

2) Fine-tuning: The process of updating a foundation model’s weights using domain-specific data. This process tends to yield better performance than using domain data alone.

3) Inference: The process of using the output of a trained model to make predictions on new data. 

4) Instruction-tuning: The process of fine-tuning an LLM on instruction ↔ response data. This is what makes LLMs good at following human instructions. 

5) Embedding: A vector representation of data (text, image, etc.). This representation captures semantic similarities between data points. 

6) Prompt / Prompt engineering: A prompt is the input to an LLM or an LLM product (ex. ChatGPT). Prompt engineering is the process of craf

In [23]:
# # delete files (after fine-tuning is done)
# client.files.delete(train_file.id)
# client.files.delete(valid_file.id)